In [11]:
import pandas as pd
import project_helper

In [12]:
df = pd.read_csv("data/cleaned_data_recent_only.csv", index_col="NPA")

In [ ]:
target = df["Employment_Rate-2022"]

features = df.drop(columns="Employment_Rate-2022")

df

,Area-2020,Age_of_Residents-2021,Tree_Canopy-2012,Impervious_Surface-2023,Housing_Density-2023,Housing_Size-2023,Housing_Age-2023,New_Residential-2023,Residential_Renovation-2023,Commuters_Driving_Alone-2022,...,Park_Proximity-2023,Job_Density-2019,Home_Sale_Price-2023,Natural_Gas_Consumption-2013,Fire_Call_Rate-2021,Fincancial_Services_Proximity-2023,Public_Nutrition_Assistance-2023,Public_Health_Insurance -2017,Subsidized_Housing-2023,Residential_Demolitions-2023
NPA,,,,,,,,,,,,,,,,,,,,,
2,410.557,31.0,56.157441,23.433162,2.8,1720.0,70.0,1.2,4.1,86.977887,...,59.066901,2.460000,488364.0,43.0,49.000000,24.471831,NaN,15.0,4.428698,2.6
3,1156.150,33.0,41.978081,38.166350,8.4,2807.0,73.0,17.6,7.1,76.234365,...,91.477741,34.330000,667092.0,49.0,63.690000,100.000000,NaN,6.0,3.889542,0.6
4,329.242,43.1,65.987392,21.782062,1.2,4158.0,45.0,2.1,7.8,90.712074,...,1.466993,0.280000,1493043.0,81.0,49.660000,15.158924,NaN,1.0,0.000000,7.3
5,167.141,32.9,42.173622,21.546276,2.1,1195.0,61.0,31.1,0.6,100.000000,...,100.000000,1.160000,255031.0,53.0,45.340000,18.965517,NaN,43.0,0.000000,2.9
6,403.223,38.0,43.716921,25.103400,2.0,1301.0,67.0,5.2,3.2,60.084034,...,79.829891,1.360000,445698.0,51.0,69.000000,69.866343,NaN,38.0,0.000000,8.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,373.419,52.3,50.560590,18.343223,1.6,3325.0,28.0,0.8,1.8,78.224101,...,69.011725,0.160000,1100345.0,59.0,NaN,9.715243,NaN,2.0,0.000000,0.0
473,494.455,43.9,50.075170,22.445100,2.4,1938.0,40.0,5.7,2.4,72.263451,...,88.841202,8.840000,485542.0,NaN,NaN,73.047210,NaN,NaN,9.668756,0.9
474,481.474,43.4,44.560229,25.384978,2.5,2589.0,31.0,0.4,1.6,96.083231,...,36.065574,1.190000,798000.0,51.0,NaN,69.098361,NaN,4.0,0.000000,0.0


In [14]:
df = project_helper.standardize_df(df)

/home/jrhol/school/dtsc-2302/dtsc-2302-final-project/.venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [15]:
df

,Area-2020,Age_of_Residents-2021,Tree_Canopy-2012,Impervious_Surface-2023,Housing_Density-2023,Housing_Size-2023,Housing_Age-2023,New_Residential-2023,Residential_Renovation-2023,Commuters_Driving_Alone-2022,...,Park_Proximity-2023,Job_Density-2019,Home_Sale_Price-2023,Natural_Gas_Consumption-2013,Fire_Call_Rate-2021,Fincancial_Services_Proximity-2023,Public_Nutrition_Assistance-2023,Public_Health_Insurance -2017,Subsidized_Housing-2023,Residential_Demolitions-2023
NPA,,,,,,,,,,,,,,,,,,,,,
2,-0.418097,-0.858584,0.599348,0.326066,0.157465,-0.586356,1.876569,-0.240473,0.264299,0.224716,...,-0.207756,0.025873,0.049995,-0.602641,0.204785,-0.603198,NaN,-0.057741,0.201382,0.796587
3,0.557507,-0.588272,-0.513685,1.860837,2.768740,0.807002,2.053879,0.562732,0.941271,-0.647024,...,0.791253,6.273766,0.708942,-0.056074,0.597797,1.819447,NaN,-0.778879,0.136022,-0.171605
4,-0.524497,0.776807,1.370966,0.154070,-0.588614,2.538766,0.398982,-0.196394,1.099232,0.527712,...,-1.983175,-0.401501,3.754119,2.858951,0.222443,-0.901920,NaN,-1.179511,-0.335489,3.071840
5,-0.736605,-0.601787,-0.498335,0.129508,-0.168945,-1.259321,1.344637,1.223907,-0.525501,1.281344,...,1.053937,-0.228983,-0.810275,0.308304,0.106867,-0.779819,NaN,2.185799,-0.335489,0.941816
6,-0.427693,0.087510,-0.377192,0.500057,-0.215574,-1.123447,1.699258,-0.044569,0.061208,-1.957479,...,0.432227,-0.189774,-0.107310,0.126115,0.739859,0.852878,NaN,1.785167,-0.335489,3.652756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,-0.466691,2.020246,0.160014,-0.204158,-0.402094,1.470995,-0.605777,-0.260063,-0.254713,-0.485575,...,0.098776,-0.425026,2.306291,0.854871,NaN,-1.076531,NaN,-1.099384,-0.335489,-0.462063
473,-0.308317,0.884933,0.121910,0.223139,-0.029055,-0.306915,0.103464,-0.020081,-0.119318,-0.969228,...,0.709986,1.276628,0.039590,NaN,NaN,0.954908,NaN,NaN,0.836609,-0.026376
474,-0.325302,0.817354,-0.310995,0.529389,0.017575,0.527562,-0.428467,-0.279653,-0.299844,0.963533,...,-0.916733,-0.223102,1.191583,0.126115,NaN,0.828244,NaN,-0.939131,-0.335489,-0.462063
